In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../data/raw/store_kpi.csv') # Load the dataset
df.head() # Display the first few rows of the dataset

# Data Cleaning
df.info() # Check for missing values
df.isnull().sum() # Count missing values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61322 entries, 0 to 61321
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   dealer_id                61322 non-null  object 
 1   month                    61322 non-null  object 
 2   total_ad_spend           61322 non-null  float64
 3   search_ad_spend          61322 non-null  float64
 4   display_ad_spend         61322 non-null  float64
 5   social_ad_spend          61322 non-null  float64
 6   video_ad_spend           61322 non-null  float64
 7   pmax_ad_spend            61322 non-null  float64
 8   search_ad_impressions    61322 non-null  float64
 9   display_ad_impressions   61322 non-null  float64
 10  social_ad_impressions    61322 non-null  float64
 11  video_ad_impressions     61322 non-null  float64
 12  pmax_ad_impressions      61322 non-null  float64
 13  search_ad_clicks         61322 non-null  float64
 14  display_ad_clicks     

dealer_id                  0
month                      0
total_ad_spend             0
search_ad_spend            0
display_ad_spend           0
social_ad_spend            0
video_ad_spend             0
pmax_ad_spend              0
search_ad_impressions      0
display_ad_impressions     0
social_ad_impressions      0
video_ad_impressions       0
pmax_ad_impressions        0
search_ad_clicks           0
display_ad_clicks          0
social_ad_clicks           0
video_ad_clicks            0
pmax_ad_clicks             0
search_ad_visits           0
display_ad_visits          0
social_ad_visits           0
video_ad_visits            0
pmax_ad_visits             0
website_visits             0
website_return_visits      0
website_new_vdps           0
website_new_searches       0
website_engagement_rate    0
website_engagements        0
website_action_rate        0
website_actions            0
lead_volume                0
unique_lead_count          0
lead_close_rate_30day      0
lead_response_

In [ ]:
# Column month to datetime
df['month'] = pd.to_datetime(df['month'])
# Column Year
df['year'] = df['month'].dt.year
# Column Month
df['month_num'] = df['month'].dt.month
df['quarter'] = df['month'].dt.quarter

print(f"Time range: {df['month'].min()} to {df['month'].max()}")
print(f"Unique dealer count: {df['dealer_id'].nunique()}")


Time range: 2023-10-01 00:00:00+00:00 to 2025-09-01 00:00:00+00:00
Unique dealer count: 2557


In [12]:
# Key Metrics
# 1. Ad Spend
# 2. Website Visits
# 3. Leads
# 4. Sales

key_metrics = ['total_ad_spend', 'website_visits', 'lead_volume', 'sales']
# Calculate monthly totals for each key metric
monthly_totals = df.groupby(['year', 'month_num'])[key_metrics].sum()

# Calculate quarterly totals for each key metric
quarterly_totals = df.groupby(['year', 'quarter'])[key_metrics].sum()

# Calculate yearly totals for each key metric
yearly_totals = df.groupby('year')[key_metrics].sum()

print("Key metrics:")
df[key_metrics].describe()






Key metrics:


,total_ad_spend,website_visits,lead_volume,sales
count,61322.000000,61322.000000,61322.000000,61322.000000
mean,4070.355625,14239.716366,42.283438,28.131274
std,9557.537942,15556.409717,63.104217,30.419416
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3608.500000,7.000000,8.000000
50%,103.675000,9737.000000,22.000000,19.000000
75%,4545.307500,19772.500000,54.000000,39.000000
max,868687.300000,226610.000000,1406.000000,438.000000


In [13]:
print(monthly_totals)
print(quarterly_totals)
print(yearly_totals)

                total_ad_spend  website_visits  lead_volume    sales
year month_num                                                      
2023 10           0.000000e+00      31520084.0     105622.0  75357.0
     11           0.000000e+00      30749058.0     111144.0  73092.0
     12           0.000000e+00      33568534.0     116354.0  76434.0
2024 1            0.000000e+00      39332124.0     110008.0  63861.0
     2            0.000000e+00      37193894.0     106124.0  63049.0
     3            0.000000e+00      38405552.0     118451.0  76111.0
     4            0.000000e+00      35587545.0     104913.0  69284.0
     5            0.000000e+00      33834689.0     109326.0  78021.0
     6            1.058253e+07      33201377.0     106327.0  68334.0
     7            1.284390e+07      34563338.0     101267.0  68436.0
     8            1.236029e+07      34701546.0     109777.0  74121.0
     9            1.275385e+07      33690231.0     101617.0  72179.0
     10           1.823180e+07    

In [16]:
# Check zero ad spend
zero_ad_spend = df[df['total_ad_spend'] == 0]
print(f"Zero ad spend: {len(zero_ad_spend)}")
print(f"Zero ad spend percentage: {len(zero_ad_spend) / len(df) * 100:.2f}%")

with_ad_spend = df[df['total_ad_spend'] > 0]
print(f"With ad spend: {len(with_ad_spend)}")
print(f"With ad spend percentage: {len(with_ad_spend) / len(df) * 100:.2f}%")

# Compare sales with ad spend and without ad spend
print(f"\nAVG sales with ad spend: {with_ad_spend['sales'].mean():.2f}")
print(f"AVG sales without ad spend: {zero_ad_spend['sales'].mean():.2f}")


Zero ad spend: 28796
Zero ad spend percentage: 46.96%
With ad spend: 32526
With ad spend percentage: 53.04%

AVG sales with ad spend: 32.69
AVG sales without ad spend: 22.99


In [ ]:
# Create derived metrics

# Total Impressions
df['total_impressions'] = (df['search_ad_impressions'] + 
                          df['display_ad_impressions'] + 
                          df['social_ad_impressions'] + 
                          df['video_ad_impressions'] + 
                          df['pmax_ad_impressions'])

# Total Clicks
df['total_clicks'] = (df['search_ad_clicks'] + 
                      df['display_ad_clicks'] + 
                      df['social_ad_clicks'] + 
                      df['video_ad_clicks'] + 
                      df['pmax_ad_clicks'])

# CTR
df['ctr'] = np.where(df['total_impressions'] > 0, 
                    df['total_clicks'] / df['total_impressions'], 0)

# Conversion Rate
df['conversion_rate'] = np.where(df['website_visits'] > 0,
                                df['unique_lead_count'] / df['website_visits'], 0)

# Return on Ad Spend (ROAS)
df['roas'] = np.where(df['total_ad_spend'] > 0,
                     df['sales'] / df['total_ad_spend'], 0)



In [21]:
# Check new derived metrics
metrics_to_check = ['total_impressions', 'total_clicks', 'ctr', 'conversion_rate', 'roas']
print("=== New derived metrics ===")
print(df[metrics_to_check].describe())

=== New derived metrics ===
       total_impressions  total_clicks           ctr  conversion_rate  \
count       6.132200e+04  6.132200e+04  61322.000000     61322.000000   
mean        2.786104e+05  3.882361e+03      0.285600         0.003479   
std         9.031047e+05  1.467906e+04     52.139270         0.029195   
min         0.000000e+00  0.000000e+00      0.000000         0.000000   
25%         0.000000e+00  0.000000e+00      0.000000         0.001163   
50%         0.000000e+00  2.900000e+01      0.000000         0.002096   
75%         2.014455e+05  3.520000e+03      0.015078         0.003444   
max         8.352077e+07  2.691666e+06  12842.000000         3.000000   

               roas  
count  61322.000000  
mean       0.643156  
std       52.514714  
min        0.000000  
25%        0.000000  
50%        0.001138  
75%        0.005980  
max    11400.000000  


In [22]:
with_ad_spend = df[df['total_ad_spend'] > 0]
print(f"\n=== With ad spend ===")
print(f"Record: {len(with_ad_spend)}")
print(f"AVG CTR: {with_ad_spend['ctr'].mean():.4f}")
print(f"AVG Conversion Rate: {with_ad_spend['conversion_rate'].mean():.4f}")
print(f"AVG ROAS: {with_ad_spend['roas'].mean():.2f}")

# Conversion Rate Distribution
print(f"\nConversion Rate Distribution:")
print(f"Median: {df['conversion_rate'].median():.4f}")
print(f"75% Quantile: {df['conversion_rate'].quantile(0.75):.4f}")
print(f"90% Quantile: {df['conversion_rate'].quantile(0.9):.4f}")


=== With ad spend ===
Record: 32526
AVG CTR: 0.5384
AVG Conversion Rate: 0.0031
AVG ROAS: 1.21

Conversion Rate Distribution:
Median: 0.0021
75% Quantile: 0.0034
90% Quantile: 0.0053


In [25]:
# AVG CTR: 0.5384 is too high, need to check
# Check abnormal CTR
print("=== CTR abnormal check ===")
print(f"CTR max: {df['ctr'].max():.4f}")
print(f"CTR median: {df['ctr'].median():.4f}")

# Check high CTR
high_ctr = df[df['ctr'] > 0.1]  # CTR > 10%
print(f"\nCTR > 10% records: {len(high_ctr)}")

if len(high_ctr) > 0:
    print("\nHigh CTR records:")
    sample = high_ctr[['dealer_id', 'month', 'total_impressions', 'total_clicks', 'ctr']].head()
    print(sample)
    
    # Check if there are records with clicks > impressions
    impossible = df[df['total_clicks'] > df['total_impressions']]
    print(f"\nRecords with clicks > impressions: {len(impossible)}")
    if len(impossible) > 0:
        print("Sample of these records:")
        print(impossible[['total_impressions', 'total_clicks', 'ctr']].head())

=== CTR abnormal check ===
CTR max: 12842.0000
CTR median: 0.0000

CTR > 10% records: 1491

High CTR records:
                                dealer_id                     month  \
224  01555a1c-5b93-4717-b3df-842c96d7b520 2024-06-01 00:00:00+00:00   
227  01555a1c-5b93-4717-b3df-842c96d7b520 2024-09-01 00:00:00+00:00   
228  01555a1c-5b93-4717-b3df-842c96d7b520 2024-10-01 00:00:00+00:00   
229  01555a1c-5b93-4717-b3df-842c96d7b520 2024-11-01 00:00:00+00:00   
230  01555a1c-5b93-4717-b3df-842c96d7b520 2024-12-01 00:00:00+00:00   

     total_impressions  total_clicks       ctr  
224            28338.0        4101.0  0.144717  
227            20880.0        2615.0  0.125239  
228             5479.0        1261.0  0.230151  
229            12771.0        2512.0  0.196696  
230            11147.0        2400.0  0.215305  

Records with clicks > impressions: 1891
Sample of these records:
     total_impressions  total_clicks  ctr
263                0.0         771.0  0.0
419                

In [26]:
# Check normal and abnormal records 
normal_records = df[df['total_clicks'] <= df['total_impressions']]
abnormal_records = df[df['total_clicks'] > df['total_impressions']]

print(f"Normal records: {len(normal_records)}")
print(f"Abnormal records: {len(abnormal_records)}")

if len(normal_records) > 0:
    print(f"\nNormal records average CTR: {normal_records['ctr'].mean():.4f}")
    print(f"Normal records CTR median: {normal_records['ctr'].median():.4f}")

Normal records: 59431
Abnormal records: 1891

Normal records average CTR: 0.0139
Normal records CTR median: 0.0000


In [33]:
# Create derived metrics (df_clean)

# Step 1: Create cleaned dataset
df_clean = df.copy()

# Step 2: Recalculate derived metrics (using cleaned data)
df_clean['total_impressions'] = (df_clean['search_ad_impressions'] + 
                                df_clean['display_ad_impressions'] + 
                                df_clean['social_ad_impressions'] + 
                                df_clean['video_ad_impressions'] + 
                                df_clean['pmax_ad_impressions'])

df_clean['total_clicks'] = (df_clean['search_ad_clicks'] + 
                           df_clean['display_ad_clicks'] + 
                           df_clean['social_ad_clicks'] + 
                           df_clean['video_ad_clicks'] + 
                           df_clean['pmax_ad_clicks'])

# Step 3: Perform data cleaning
# Case 1: Zero impressions but non-zero clicks → Set clicks to 0
df_clean.loc[(df_clean['total_impressions'] == 0) & (df_clean['total_clicks'] > 0), 'total_clicks'] = 0

# Case 2: Clicks far exceed impressions → Limit CTR to reasonable range
df_clean['ctr_temp'] = np.where(df_clean['total_impressions'] > 0, 
                               df_clean['total_clicks'] / df_clean['total_impressions'], 0)

# Limit CTR to 10% maximum
high_ctr_mask = df_clean['ctr_temp'] > 0.1
df_clean.loc[high_ctr_mask, 'total_clicks'] = df_clean.loc[high_ctr_mask, 'total_impressions'] * 0.1

# Step 4: Calculate final metrics
df_clean['ctr_clean'] = np.where(df_clean['total_impressions'] > 0, 
                                df_clean['total_clicks'] / df_clean['total_impressions'], 0)
# Conversion Rate
df_clean['conversion_rate'] = np.where(df_clean['website_visits'] > 0,
                                df_clean['unique_lead_count'] / df_clean['website_visits'], 0)

# Return on Ad Spend (ROAS)
df_clean['roas'] = np.where(df_clean['total_ad_spend'] > 0,
                     df_clean['sales'] / df_clean['total_ad_spend'], 0)



In [36]:
# Re-analyze conversion funnel with cleaned data
print("=== Conversion Funnel Analysis (After Cleaning) ===")
print(f"Average impressions: {df_clean['total_impressions'].mean():.0f}")
print(f"Average clicks: {df_clean['total_clicks'].mean():.0f}")
print(f"Average website visits: {df_clean['website_visits'].mean():.0f}")
print(f"Average leads: {df_clean['unique_lead_count'].mean():.0f}")
print(f"Average sales: {df_clean['sales'].mean():.0f}")

# Recalculate conversion rates (using cleaned data)
df_clean['impression_to_click'] = np.where(df_clean['total_impressions'] > 0,
                                          df_clean['total_clicks'] / df_clean['total_impressions'], 0)
df_clean['click_to_visit'] = np.where(df_clean['total_clicks'] > 0,
                                     df_clean['website_visits'] / df_clean['total_clicks'], 0)
df_clean['visit_to_unique_lead'] = np.where(df_clean['website_visits'] > 0,
                                    df_clean['unique_lead_count'] / df_clean['website_visits'], 0)
df_clean['visit_to_lead'] = np.where(df_clean['website_visits'] > 0,
                                    df_clean['lead_volume'] / df_clean['website_visits'], 0)
df_clean['lead_to_sale'] = np.where(df_clean['unique_lead_count'] > 0,
                                   df_clean['sales'] / df_clean['unique_lead_count'], 0)

print(f"\nStage conversion rates:")
print(f"Impressions → Clicks: {df_clean['impression_to_click'].mean():.4f} ({df_clean['impression_to_click'].mean()*100:.2f}%)")
print(f"Clicks → Visits: {df_clean['click_to_visit'].mean():.4f} ({df_clean['click_to_visit'].mean()*100:.2f}%)")
print(f"Visits → Leads: {df_clean['visit_to_lead'].mean():.4f} ({df_clean['visit_to_lead'].mean()*100:.2f}%)")
print(f"Visits → Leads (unique): {df_clean['visit_to_unique_lead'].mean():.4f} ({df_clean['visit_to_unique_lead'].mean()*100:.2f}%)")
print(f"Leads → Sales: {df_clean['lead_to_sale'].mean():.4f} ({df_clean['lead_to_sale'].mean()*100:.2f}%)")

=== Conversion Funnel Analysis (After Cleaning) ===
Average impressions: 278610
Average clicks: 3819
Average website visits: 14240
Average leads: 36
Average sales: 28

Stage conversion rates:
Impressions → Clicks: 0.0113 (1.13%)
Clicks → Visits: 39.5579 (3955.79%)
Visits → Leads: 0.0040 (0.40%)
Visits → Leads (unique): 0.0035 (0.35%)
Leads → Sales: 1.2835 (128.35%)


In [41]:
# Performance Metrics
df_clean['sales_per_visit'] = np.where(df_clean['website_visits'] > 0,
                                      df_clean['sales'] / df_clean['website_visits'], 0)

df_clean['cost_per_lead'] = np.where(df_clean['unique_lead_count'] > 0,
                                    df_clean['total_ad_spend'] / df_clean['unique_lead_count'], 0)

df_clean['leads_per_visit'] = np.where(df_clean['website_visits'] > 0,
                                      df_clean['unique_lead_count'] / df_clean['website_visits'], 0)

# Efficiency Metrics
df_clean['ad_efficiency'] = np.where(df_clean['total_ad_spend'] > 0,
                                    df_clean['total_clicks'] / df_clean['total_ad_spend'], 0)

df_clean['visit_efficiency'] = np.where(df_clean['total_clicks'] > 0,
                                       df_clean['website_visits'] / df_clean['total_clicks'], 0)

# Channel Mix Features (important for clustering)
df_clean['search_spend_ratio'] = np.where(df_clean['total_ad_spend'] > 0,
                                         df_clean['search_ad_spend'] / df_clean['total_ad_spend'], 0)

df_clean['social_spend_ratio'] = np.where(df_clean['total_ad_spend'] > 0,
                                         df_clean['social_ad_spend'] / df_clean['total_ad_spend'], 0)

df_clean['display_spend_ratio'] = np.where(df_clean['total_ad_spend'] > 0,
                                          df_clean['display_ad_spend'] / df_clean['total_ad_spend'], 0)

df_clean['video_spend_ratio'] = np.where(df_clean['total_ad_spend'] > 0,
                                        df_clean['video_ad_spend'] / df_clean['total_ad_spend'], 0)

df_clean['pmax_spend_ratio'] = np.where(df_clean['total_ad_spend'] > 0,
                                       df_clean['pmax_ad_spend'] / df_clean['total_ad_spend'], 0)

# Temporal Features
df_clean['is_holiday_season'] = df_clean['month_num'].isin([11, 12])
df_clean['is_quarter_end'] = df_clean['month_num'].isin([3, 6, 9, 12])
df_clean['is_summer'] = df_clean['month_num'].isin([6, 7, 8])

In [ ]:
dealer_features = df_clean.groupby('dealer_id').agg({
    'sales': ['sum', 'mean', 'std', 'count'],
    'total_ad_spend': ['sum', 'mean'],
    'website_visits': ['sum', 'mean'],
    'unique_lead_count': ['sum', 'mean'],
    'ctr_clean': 'mean',
    'conversion_rate': 'mean',
    'roas': 'mean',
    'sales_per_visit': 'mean',
    'cost_per_lead': 'mean',
    'leads_per_visit': 'mean',
    'ad_efficiency': 'mean',
    'visit_efficiency': 'mean',
    'search_spend_ratio': 'mean',
    'social_spend_ratio': 'mean',
    'display_spend_ratio': 'mean',
    'video_spend_ratio': 'mean',
    'pmax_spend_ratio': 'mean',
    'is_holiday_season': 'mean', 
    'is_quarter_end': 'mean',
    'is_summer': 'mean',
    'month': 'nunique'  # months active
}).round(4)

# 3. Flatten column names
dealer_features.columns = ['_'.join(col).strip() for col in dealer_features.columns]
dealer_features = dealer_features.reset_index()

# 4. Add performance tiers
dealer_features['total_sales_tier'] = pd.cut(dealer_features['sales_sum'], 
                                            bins=3, labels=['Low', 'Medium', 'High'])
dealer_features['avg_roas_tier'] = pd.cut(dealer_features['roas_mean'], 
                                         bins=3, labels=['Low', 'Medium', 'High'])
dealer_features['conversion_tier'] = pd.cut(dealer_features['conversion_rate_mean'], 
                                           bins=3, labels=['Low', 'Medium', 'High'])

print("Dealer-level features created!")
print(f"Shape: {dealer_features.shape}")
print("\nColumns created:")
print(dealer_features.columns.tolist())

Dealer-level features created!
Shape: (2557, 31)

Columns created:
['dealer_id', 'sales_sum', 'sales_mean', 'sales_std', 'sales_count', 'total_ad_spend_sum', 'total_ad_spend_mean', 'website_visits_sum', 'website_visits_mean', 'unique_lead_count_sum', 'unique_lead_count_mean', 'ctr_clean_mean', 'conversion_rate_mean', 'roas_mean', 'sales_per_visit_mean', 'cost_per_lead_mean', 'leads_per_visit_mean', 'ad_efficiency_mean', 'visit_efficiency_mean', 'search_spend_ratio_mean', 'social_spend_ratio_mean', 'display_spend_ratio_mean', 'video_spend_ratio_mean', 'pmax_spend_ratio_mean', 'is_holiday_season_mean', 'is_quarter_end_mean', 'is_summer_mean', 'month_nunique', 'total_sales_tier', 'avg_roas_tier', 'conversion_tier']


In [43]:
# Prepare features for clustering
clustering_features = [
    'sales_mean', 'total_ad_spend_mean', 'website_visits_mean',
    'unique_lead_count_mean', 'ctr_clean_mean', 'conversion_rate_mean', 'roas_mean',
    'search_spend_ratio_mean', 'social_spend_ratio_mean', 'display_spend_ratio_mean',
    'video_spend_ratio_mean', 'pmax_spend_ratio_mean'
]

print(f"\n=== Feature distributions for clustering ===")
print(dealer_features[clustering_features].describe())


=== Feature distributions for clustering ===
        sales_mean  total_ad_spend_mean  website_visits_mean  \
count  2557.000000          2557.000000          2557.000000   
mean     28.112812          4067.304583         14229.102203   
std      28.736634          6189.910753         14307.342653   
min       0.000000             0.000000             0.000000   
25%       8.250000           487.372100          4185.166700   
50%      18.666700          1956.151700         10086.666700   
75%      38.750000          5114.491200         19966.083300   
max     284.041700         84857.485800        161026.125000   

       unique_lead_count_mean  ctr_clean_mean  conversion_rate_mean  \
count             2557.000000     2557.000000           2557.000000   
mean                36.131274        0.011265              0.003477   
std                 45.997555        0.009701              0.011612   
min                  0.000000        0.000000              0.000000   
25%                  7

In [44]:
# Save all processed data
df_clean.to_csv('../data/processed/store_kpi_clean.csv', index=False)
dealer_features.to_csv('../data/processed/dealer_features.csv', index=False)

print("All processed data saved!")
print(f"Individual records: {len(df_clean)}")
print(f"Dealer features: {len(dealer_features)}")

All processed data saved!
Individual records: 61322
Dealer features: 2557
